## Sentiment analysis with hotel reviews

### Processing the data

#### Load the data

In [2]:
# Load the hotel reviews from CSV
import pandas as pd
import time
# importing time so the start and end time can be used to calculate file loading time
print("Loading data file now, this could take a while depending on file size")
start = time.time()
# df is 'DataFrame' - make sure you downloaded the file to the data folder
df = pd.read_csv('Hotel_Reviews.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")

Loading data file now, this could take a while depending on file size
Loading took 1.48 seconds


#### Dataframe operations

In [10]:
# 1. Print out the shape of the data frame you have just loaded (the shape is the number of rows and columns)
df.shape
# the followings are the columns in the dataframe
# Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
#        'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
#        'Negative_Review', 'Review_Total_Negative_Word_Counts',
#        'Total_Number_of_Reviews', 'Positive_Review',
#        'Review_Total_Positive_Word_Counts',
#        'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
#        'days_since_review', 'lat', 'lng'],
#       dtype='object')

# 2 Calculate the frequency count for reviewer nationalities
# 2.1 How many distinct values are there for the column Reviewer_Nationality and what are they?
nationality_freq = df["Reviewer_Nationality"].value_counts()
# print(nationality_freq)
# 2.2 What reviewer nationality are the top 10 in the dataset (print country and number of reviews)?
# print(nationality_freq[0:10].to_string())

# 3 What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities
for nat in nationality_freq[:10].index:
    # First, extract all the rows that match the criteria into a new dataframe
    nat_df = df[df["Reviewer_Nationality"] == nat]
    # Now get the hotel freq
    freq = nat_df["Hotel_Name"].value_counts()
    # print("[" + str(nat).strip() + "] was [" + str(freq.index[0]) + "] with [" + str(freq[0]) + "] reviews.")

# 4 How many reviews are there per hotel (frequency count of hotel) in the dataset?
nationality_freq = df["Hotel_Name"].value_counts()
# print(nationality_freq)
# print(nationality_freq[0:10].to_string())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# 5 calculate an average score. Add a new column to your dataframe with the column header Calc_Average_Score that contains that calculated average.
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)
# print(df['Calc_Average_Score'][0:10].to_string())

In [11]:
# 6 Do any hotels have the same Average_Score and Calc_Average_Score?
# define a function that takes a row and performs some calculation with it
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]
# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)
# print hotels that have the same average score
same_avg = df[df["Average_Score_Difference"] == 0]
print(same_avg[["Hotel_Name", "Average_Score", "Calc_Average_Score"]][0:10].to_string(index=False))

                      Hotel_Name  Average_Score  Calc_Average_Score
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7


In [28]:
# 7 Calculate and print out how many rows have column Negative_Review values of "No Negative"
no_negative_reviews = df[df['Negative_Review'] == 'No Negative']
# print(no_negative_reviews[['Hotel_Name', 'Negative_Review']][0:20].to_string())

# 8 Calculate and print out how many rows have column Positive_Review values of "No Positive"
no_positive_reviews = df[df['Positive_Review'] == 'No Positive']
# print(no_positive_reviews[['Hotel_Name', 'Positive_Review']][0:20].to_string())

# 9 Calculate and print out how many rows have column Negative_Review values of "No Negative" and Positive_Review values of "No Positive"
no_reviews = df[(df['Negative_Review'] == 'No Negative') & (df['Positive_Review'] == 'No Positive')]
print(no_reviews[['Hotel_Name', 'Negative_Review', 'Positive_Review']][0:20].to_string())

                                       Hotel_Name Negative_Review Positive_Review
1403                      Apex Temple Court Hotel     No Negative     No Positive
7272                Park Plaza County Hall London     No Negative     No Positive
7892                    Hotel Trianon Rive Gauche     No Negative     No Positive
21443                 Novotel London Tower Bridge     No Negative     No Positive
22770                   W London Leicester Square     No Negative     No Positive
27079                               The Grosvenor     No Negative     No Positive
27135                               The Grosvenor     No Negative     No Positive
33812                          11 Cadogan Gardens     No Negative     No Positive
40033                                  Arbor City     No Negative     No Positive
52650                    Hilton London Paddington     No Negative     No Positive
55638                             COQ Hotel Paris     No Negative     No Positive
55781           